In [67]:
TRAIN_SET = 'input/train.csv'

In [68]:
import pandas as pd
import os

def load_data(path):
    pwd = os.getcwd()
    file_path = os.path.join(pwd,path)
    return pd.read_csv(file_path)

# Create folders 

In [69]:
import pandas as pd
from sklearn import model_selection
df = pd.read_csv('input/train.csv')
df['kfold'] = -1
df = df.sample(frac=1).reset_index(drop=True)
df.head()


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,kfold
0,118726,0,0,1,F,Y,Green,Trapezoid,Lion,Russia,...,1,Novice,Lava Hot,f,G,xy,3,1,0,-1
1,159375,0,0,0,F,N,Red,Circle,Dog,Costa Rica,...,1,Grandmaster,Freezing,a,E,ut,4,2,0,-1
2,52882,0,1,1,F,N,Green,Trapezoid,Snake,India,...,1,Novice,Freezing,j,U,TZ,3,2,0,-1
3,41361,0,1,0,T,N,Blue,Circle,Lion,China,...,1,Master,Boiling Hot,j,I,od,4,7,1,-1
4,31519,0,0,0,T,N,Red,Square,Lion,Russia,...,1,Grandmaster,Lava Hot,a,J,kW,3,10,1,-1


In [70]:
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
xc = None
for fold, (train_idx,val_idx) in enumerate(kf.split(X=df,y=df.target.values)):
    print(len(train_idx),len(val_idx))
    df.loc[val_idx,'kfold'] = fold
df.to_csv('input/train_folds.csv',index=False)

240000 60000
240000 60000
240000 60000
240000 60000
240000 60000


In [71]:
df = pd.read_csv('input/train_folds.csv')
df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,kfold
0,118726,0,0,1,F,Y,Green,Trapezoid,Lion,Russia,...,1,Novice,Lava Hot,f,G,xy,3,1,0,1
1,159375,0,0,0,F,N,Red,Circle,Dog,Costa Rica,...,1,Grandmaster,Freezing,a,E,ut,4,2,0,0
2,52882,0,1,1,F,N,Green,Trapezoid,Snake,India,...,1,Novice,Freezing,j,U,TZ,3,2,0,1
3,41361,0,1,0,T,N,Blue,Circle,Lion,China,...,1,Master,Boiling Hot,j,I,od,4,7,1,1
4,31519,0,0,0,T,N,Red,Square,Lion,Russia,...,1,Grandmaster,Lava Hot,a,J,kW,3,10,1,2


In [72]:
df.kfold.value_counts()

1    60000
0    60000
2    60000
4    60000
3    60000
Name: kfold, dtype: int64

# training

In [90]:
import os

import pandas as pd
from matplotlib.pyplot import axis
from sklearn import ensemble, metrics, preprocessing
import numpy as np 
from matplotlib import pyplot as plt 


FOLD = 0
FOLD_MAPPPING = {
    0: [1, 2, 3, 4],
    1: [0, 2, 3, 4],
    2: [0, 1, 3, 4],
    3: [0, 1, 2, 4],
    4: [0, 1, 2, 3]
}
df = pd.read_csv('input/train_folds.csv')
train_df = df[df.kfold.isin(FOLD_MAPPPING.get(FOLD))]
valid_df = df[df.kfold == FOLD]

ytrain = train_df.target.values
yvalid = valid_df.target.values

train_df = train_df.drop(['id','target','kfold'],axis=1)
valid_df = valid_df.drop(['id','target','kfold'],axis=1)

valid_df = valid_df[train_df.columns]

labels_encoders = []

for c in train_df.columns:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(train_df[c].values.tolist() + valid_df[c].values.tolist())
    train_df.loc[:, c] = lbl.transform(train_df[c].values.tolist())
    valid_df.loc[:, c] = lbl.transform(valid_df[c].values.tolist())
    labels_encoders.append((c,lbl))

data is ready to train

clf = ensemble.RandomForestClassifier(n_jobs=-1,verbose=2,n_estimators=9)
clf.fit(train_df,ytrain)
preds = clf.predict_proba(valid_df)[:,1]
print(metrics.roc_auc_score(yvalid,preds))